In [1]:
import numpy as np
import pandas as pd
import os, sys
from pathlib import Path
from hist import intervals

sys.path.append("/users/hep/eigen1907/Workspace/Workspace-RPC/modules")
from NanoAODTnP.Plotting.DataLoader import DataLoader
from NanoAODTnP.Plotting.Hist1D import init_figure

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
from hist import intervals
from datetime import datetime

def run2time(run, run_info):
    time = None
    while True:
        if time is not None: break
        if round(run) in run_info['run_number'].values:
            time = run_info['start_time'][run_info['run_number'] == round(run)].values[0]
            time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        run += 1
    return time

def plot_eff_time(ax, data, run_info):
    tree = data.tree 
    runs = np.unique(tree['run'])
    times = []
    effs, err_uppers, err_lowers = [], [], []
    for run in runs:
        times.append(run2time(run, run_info))
        run_mask = (tree['run'] == run)

        total = tree['is_matched'][run_mask]
        passed = total[total]

        eff = len(passed) / len(total) * 100
        err = intervals.clopper_pearson_interval(len(passed), len(total), 0.68) * 100
        
        effs.append(eff)
        err_lowers.append(eff - err[0])
        err_uppers.append(err[1] - eff)
        
        #if len(total) < 1000:
        #    print(f"Run: {run}, nTotal: {len(total)}, Lower Bound: {err[0]}")

    ax.errorbar(
        #runs,
        times,
        effs,
        yerr = (err_lowers, err_uppers),
        fmt = 's',
        markersize = 6,
        markerfacecolor = 'none',
        markeredgewidth = 2,
        lw = 2,
        capsize = 4,
        color = data.facecolors[1],
        label = data.region,
        alpha = 1.0
    )
    return ax, effs, runs

In [3]:
def plot_eff_time_2022(data_2022, region, output_dir):
    fig, ax = init_figure(
        figsize = (20, 9),
        fontsize = 24,
        com = 13.6,
        label1 = 'Work in Progress',
        label2 = f'2022 Data',
        mid_label = f'RPC {region}',
        loc = 0,
        ylabel = 'Efficiency [%]',
        xlim = None,
        ylim = None,
        xticks = None,
        yticks = None,
        log_scale = False,
    )
    
    ymin, ymax = 0, 100
    ax.set_xlim(datetime.strptime('2022/01/Jul', '%Y/%d/%b'), datetime.strptime('2022/01/Dec', '%Y/%d/%b'))
    ax.set_ylim(ymin, ymax)
    
    ax, effs, runs = plot_eff_time(ax, data_2022.filter_data(region = region), run_info)
    
    spans = [
        (355100, 355769, 'Run2022B', 'y'),
        (355862, 357482, 'Run2022C', 'm'),
        (357538, 357900, 'Run2022D', 'y'),
        (359356, 360327, 'Run2022E', 'm'),
        (360335, 362167, 'Run2022F', 'y'),
        (362362, 362760, 'Run2022G', 'm'),
    ]
    
    for start, end, label, color in spans:
        ax.axvspan(run2time(start, run_info), run2time(end, run_info), color=color, alpha=0.05)
        mid_point = run2time((start + end) // 2, run_info)
        ax.text(mid_point, ymin + 15, label, rotation=90, verticalalignment='center', fontsize=22, weight='bold', color=color)
    
    #ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=(1, 15)))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%b'))
    ax.grid()
    
    #ax.legend(loc='center right', fontsize = 28)
    
        
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    fig.savefig(output_dir / f"{region}.png")
    plt.close(fig)


def plot_eff_time_2023(data_2023, region, output_dir):
    fig, ax = init_figure(
        figsize = (20, 9),
        fontsize = 24,
        com = 13.6,
        label1 = 'Work in Progress',
        label2 = f'2023 Data',
        mid_label = f'RPC {region}',
        loc = 0,
        ylabel = 'Efficiency [%]',
        xlim = None,
        ylim = None,
        xticks = None,
        yticks = None,
        log_scale = False,
    )
    
    ymin, ymax = 0, 100
    ax.set_xlim(datetime.strptime('2023/15/Apr', '%Y/%d/%b'), datetime.strptime('2023/01/Aug', '%Y/%d/%b'))
    ax.set_ylim(ymin, ymax)
    
    ax, effs, runs = plot_eff_time(ax, data_2023.filter_data(region = region), run_info)
    
    spans = [
        (366403, 367079, 'Run2023B', 'y'),
        (367770, 369694, 'Run2023C', 'm'),
        (370616, 371225, 'Run2023D', 'y'),
    ]
    
    for start, end, label, color in spans:
        ax.axvspan(run2time(start, run_info), run2time(end, run_info), color=color, alpha=0.05)
        mid_point = run2time((start + end) // 2, run_info)
        ax.text(mid_point, ymin + 15, label, rotation=90, verticalalignment='center', fontsize=22, weight='bold', color=color)
    
    #ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=(1, 15)))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%b'))
    ax.grid()
    
    #ax.legend(loc='center right', fontsize = 28)
    
    #plt.tight_layout(rect=[0, 0, 1, 1])
    #plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
        
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    fig.savefig(output_dir / f"{region}.png")
    plt.close(fig)


In [4]:
workspace = Path('/users/hep/eigen1907/Workspace/Workspace-RPC')

input_2022_path = workspace / 'data' / 'efficiency' / 'data' / 'Run2022.root'
roll_blacklist_2022_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'roll-blacklist-2022.json'

input_2023_path = workspace / 'data' / 'efficiency' / 'data' / 'Run2023.root'
roll_blacklist_2023_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'roll-blacklist-2023.json'

run_blacklist_path = workspace / 'data' / 'efficiency' / 'blacklist' / 'run-blacklist.json'
geom_path = workspace / 'data' / 'efficiency' / 'geometry' / 'run3.csv'

run_info = pd.read_csv("/users/hep/eigen1907/Workspace/Workspace-RPC/data/efficiency/run_info/run_info.csv", index_col=False)

data_2022 = DataLoader(
    input_path = input_2022_path,
    geom_path = geom_path,
    roll_blacklist_path = roll_blacklist_2022_path,
    run_blacklist_path = run_blacklist_path,
    var = ['run']
)

data_2023 = DataLoader(
    input_path = input_2023_path,
    geom_path = geom_path,
    roll_blacklist_path = roll_blacklist_2023_path,
    run_blacklist_path = run_blacklist_path,
    var = ['run']
)

/users/hep/eigen1907/micromamba/envs/ds4hep/lib/python3.11/site-packages/hist/basehist.py:400: UserWarning: List indexing selection is experimental. Removed bins are not placed in overflow.
  return super().__getitem__(self._index_transform(index))
/users/hep/eigen1907/micromamba/envs/ds4hep/lib/python3.11/site-packages/hist/basehist.py:400: UserWarning: List indexing selection is experimental. Removed bins are not placed in overflow.
  return super().__getitem__(self._index_transform(index))


In [5]:
fiducial_data_2022 = data_2022.filter_data(keys = ['is_fiducial'])
fiducial_data_2023 = data_2023.filter_data(keys = ['is_fiducial'])

filter_by_roll_data_2022 = data_2022.filter_data(keys = ['is_fiducial', 'is_linked'])
filter_by_roll_data_2023 = data_2023.filter_data(keys = ['is_fiducial', 'is_linked'])

filter_by_roll_run_data_2022 = data_2022.filter_data(keys = ['is_fiducial', 'is_linked', 'is_safetime'])
filter_by_roll_run_data_2023 = data_2023.filter_data(keys = ['is_fiducial', 'is_linked', 'is_safetime'])

In [6]:
regions = [
    'All', 'Barrel', 'Endcap', 'Disk123', 'Disk4',
    'W-2', 'W-1', 'W+0', 'W+1', 'W+2',
    'RE+1', 'RE+2', 'RE+3', 'RE+4',
    'RE-1', 'RE-2', 'RE-3', 'RE-4',
]

workspace = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/Plotting/Scatter-run-eff/by_region/')

for region in regions:
    plot_eff_time_2022(fiducial_data_2022, region, workspace / 'no_filter_2022')
    plot_eff_time_2022(filter_by_roll_data_2022, region, workspace / 'filter_by_roll_2022')
    plot_eff_time_2022(filter_by_roll_run_data_2022, region, workspace / 'filter_by_roll_run_2022')
    plot_eff_time_2023(fiducial_data_2023, region, workspace / 'no_filter_2023')
    plot_eff_time_2023(filter_by_roll_data_2023, region, workspace / 'filter_by_roll_2023')
    plot_eff_time_2023(filter_by_roll_run_data_2023, region, workspace / 'filter_by_roll_run_2023')